In [414]:
#### 查看20newsgroups

In [415]:
from sklearn.datasets import fetch_20newsgroups
# 20newsgroup 是按照时间序列进行分割测试集和训练集的
import numpy as np

In [416]:
rec = ['rec.autos','rec.motorcycles','rec.sport.baseball','rec.sport.hockey']
talk = ['talk.politics.guns','talk.politics.mideast','talk.politics.misc','talk.religion.misc']

In [417]:
ng_train_rec = fetch_20newsgroups(subset='train',categories = rec)
ng_train_talk = fetch_20newsgroups(subset='train',categories = talk)

In [418]:
ng_test_rec = fetch_20newsgroups(subset='test',categories = rec)
ng_test_talk= fetch_20newsgroups(subset='test',categories = talk)

- np.concatenate 比 np.append 在拼接数组上更加高效?
- np.append 只能进行两个数组的拼接，而np.concatenate 可以进行多个数组的拼接


In [419]:
# 验证程序，至于为什么，有待验证

In [420]:
from time import clock as now
a =  np.array(range(100000000))
b =  np.array(range(100000000))
time1 = now()
all = np.append(a,b)
time2 = now()
print(time2 - time1)

1.3609109999999873


In [421]:
time1 = now()
all = np.concatenate((a,b))
time2 = now()
print(time2 - time1)

1.0961079999999583


In [500]:
target_name = ng_train_rec.target_names
ng_rec = ng_train_rec.data + ng_test_rec.data
ng_talk = ng_train_talk.data + ng_test_talk.data
ng_rec_target = np.concatenate((ng_train_rec.target,ng_test_rec.target))
ng_talk_target = np.concatenate((ng_train_talk.target,ng_test_talk.target))

In [501]:
ng_rec[:1]

['From: swdwan@napier.uwaterloo.ca (Donald Wan)\nSubject: $ 80 SVX OIL CHANGE\n \nOrganization: University of Waterloo\nLines: 17\n\n\n\n\n\n My friend brought a subaru SVX recently.  I had drove it for couples times and I\nthink its a great car, esp on snow.  However when she took it to a local Subaru\ndealer for a oil change, the bill came out to be about 80 dollars.  The dealer\ntold us it is because to change the oil filter on a SVX it is necessary to\ndisassemble a metal cover under the engine and that took an hour of labour.\nAt first, we think we are being ripped off so she phone to a dealer in Toronto\nbut found out the they are charging roughly the same price.  So is there any\nSVX owner out there that has the same problem ?  And if the oil change story is\ntrue, then the engineer of Subaru looks pretty stubid to me. By the way, the car\nlooks great.\n\nSWD Wan.\n\n']

In [503]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import scipy.stats as ss
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
import nltk
import re

In [609]:
# KL 使用二元bigram 来计算一个句子出现的概率,句子概率计算的概率如下：

![bigram](img/bigram.png)

In [504]:
one_rec_dict = defaultdict(int) # 一个元素 res
two_rec_dict = defaultdict(int) # 两个元素
one_talk_dict = defaultdict(int) # 
two_talk_dict = defaultdict(int) 
from nltk.corpus import stopwords
# 使用停用词
stopworddic = set(stopwords.words('english'))
stopword = open('../../../tool/text/stopword_en.txt').read().split('\n')
for word in stopword:
    stopworddic.add(word)
# 
pattern = '[\n\t_*\?\"-=\[\],<>:()\.\\!/\| ]+|[\d]+' 
ng_rec_result = []
ng_talk_result = []
for ng_rec_i in ng_rec:
    clean_data = re.split(pattern,ng_rec_i)
    clean_data = [i for i in clean_data if i not in stopworddic ]
    ng_rec_result.append(' '.join(clean_data))

for ng_talk_i in ng_talk:
    clean_data = re.split(pattern,ng_talk_i)
    clean_data = [i for i in clean_data if i not in stopworddic]
    ng_talk_result.append(' '.join(clean_data))

In [508]:
len(ng_rec_result)

3979

In [610]:
# 只取tfidf 值的前200个值
ng_rec_final = []
ng_talk_final = []
vectorizer = TfidfVectorizer()
transform = vectorizer.fit_transform(ng_rec_result)
feature_names = vectorizer.get_feature_names()
for trans in transform.toarray():
    index = np.argsort(trans)[:]
    ng_rec_final.append(np.array(feature_names)[index])
transform = vectorizer.fit_transform(ng_talk_result)
feature_names = vectorizer.get_feature_names()
for trans in transform.toarray():
    index = np.argsort(trans)[:]
    ng_talk_final.append(np.array(feature_names)[index])

In [611]:
for ng_rec_i in ng_rec_final:
    for i in range(len(ng_rec_i)):
        if len(ng_rec_i[i]) == 1:
            continue
        one_rec_dict[ng_rec_i[i]] += 1
for ng_talk_i in ng_talk_final:
    for i in range(len(ng_talk_i)):
        if len(ng_talk_i[i]) == 1:
            continue
        one_talk_dict[ng_talk_i[i]] += 1

In [612]:
# 统计总的词数
one_rec_count = 0
one_talk_count = 0
for val in one_rec_dict.values():
    one_rec_count += val
for val in one_talk_dict.values():
    one_talk_count += val

In [613]:
# 使用2-gram的话，在一种出现的根本不会在二中出现
rec_prob = []
talk_prob = []
for ng_rec_i in ng_rec_final:
    p1 = 0; p2 = 0
    flag = 0; 
    for i in range(len(ng_rec_i)):
        if len(ng_rec_i[i]) == 1:
            continue
        p1 +=  np.log(one_rec_dict[ng_rec_i[i]] / one_rec_count)
        if one_talk_dict[ng_rec_i[i]] == 0:
            p2 = 0;
            flag = 1
        if flag == 0:
            p2 += np.log(one_talk_dict[ng_rec_i[i]] / one_talk_count)
    rec_prob.append(p1)
    talk_prob.append(p2)

In [615]:
talk_prob = np.array(talk_prob)

In [622]:
# res ^ talk 的KL 散度比较 
talk_prob = talk_prob + 1e-12  # 出现零值时，对其的修正
# rec_prob /= np.sum(rec_prob)
# talk_prob /= np.sum(talk_prob)
ss.entropy(rec_prob,talk_prob)

0.0002513194279917065

## conclusion
- 使用KL散度确实可以比较两个分布的相关性，数值越小，相关性越高，论文中得到的结果还没有我的好，不是不我做错了！
- 两个类别之间的KL散度真的好难控制，因为只要有一个词不在对方的词库里面，那么其概率就是为0，就完全否定了相关性。
- 使用1-gram 就已经很难去控制，更不要说使用2-gram了